<a href="https://colab.research.google.com/github/JVengava/CS6910-DeepLearningAssignments/blob/main/Assignment3/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
pip install wandb

     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 144 kB 52.0 MB/s 
     |████████████████████████████████| 181 kB 48.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=5672a63678222f6023d7162f1b563c0b96347b1f4e0fd96c5e0ee7a5c71b3891
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [9]:
!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf dakshina_dataset_v1.0.tar

--2022-05-08 17:49:45--  https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 173.194.217.128, 74.125.31.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2008340480 (1.9G) [application/x-tar]
Saving to: ‘dakshina_dataset_v1.0.tar’

dakshina_dataset_v1 100%[===================>]   1.87G   234MB/s    in 7.8s    

2022-05-08 17:49:53 (247 MB/s) - ‘dakshina_dataset_v1.0.tar’ saved [2008340480/2008340480]



In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import pathlib


class DataProcessor():

  def __init__(self, datapath, input_language = 'en', ouput_language = "hi"):
  
    self.input_language = input_language
    self.ouput_language = ouput_language

    self.trainpath = os.path.join(datapath, ouput_language, "lexicons", ouput_language+".translit.sampled.train.tsv")
    self.valpath = os.path.join(datapath, ouput_language, "lexicons", ouput_language+".translit.sampled.dev.tsv")
    self.testpath = os.path.join(datapath, ouput_language, "lexicons", ouput_language+".translit.sampled.test.tsv")

    self.train = pd.read_csv(self.trainpath, sep="\t", names=["tgt", "src", "count"])
    self.val = pd.read_csv(self.valpath, sep="\t", names=["tgt", "src", "count"])
    self.test = pd.read_csv(self.testpath, sep="\t", names=["tgt", "src", "count"])

    # Train data
    self.train_data = self.ProcessData(self.train["src"].to_list(), self.train["tgt"].to_list())
    (self.input_words, self.output_words, self.input_chars, self.output_chars) = self.train_data

    self.input_token_index_map = dict([(char, i) for i, char in enumerate(self.input_chars)])
    self.output_token_index_map = dict([(char, i) for i, char in enumerate(self.output_chars)])

    (self.train_encoder_input,self.train_decoder_input,self.train_decoder_target) = self.encode(self.input_words, self.output_words, self.input_chars, self.output_chars, self.input_token_index_map, self.output_token_index_map)
    
    # Validation data            
    self.val_encoder_input, self.val_decoder_input, self.val_decoder_target = self.encode(self.val["src"].to_list(), self.val["tgt"].to_list(), list(self.input_token_index_map.keys()), list(self.output_token_index_map.keys()), self.input_token_index_map, self.output_token_index_map)        

    # Test data    
    self.test_encoder_input, self.test_decoder_input, self.test_decoder_target = self.encode(self.test["src"].to_list(), self.test["tgt"].to_list(), list(self.input_token_index_map.keys()), list(self.output_token_index_map.keys()), self.input_token_index_map, self.output_token_index_map)
        		    

  def ProcessData(self, input , output):
    input_words = []
    output_words = []

    input_chars = set()
    output_chars = set()        
    
    input = [str(x) for x in input]
    target = [str(x) for x in output]

    for input_text, output_text in zip(input, output):
        output_text = "\t" + output_text + "\n"
        input_words.append(input_text)
        output_words.append(output_text)          
        for char in input_text:
            if char not in input_chars:
                input_chars.add(char)      
        for char in output_text:
            if char not in output_chars:
                output_chars.add(char)

    input_chars = sorted(list(input_chars))
    output_chars = sorted(list(output_chars))

    input_chars.append(" ")
    output_chars.append(" ")

    return input_words, output_words, input_chars, output_chars


  def encode(self, input_words, output_words, input_chars, output_chars, input_token_index_map, output_token_index_map):
        
    print("Number of samples:", len(input_words))
    
    num_encoder_tokens = len(input_chars)
    num_decoder_tokens = len(output_chars)		
    print("Number of unique input tokens:"
    , num_encoder_tokens)
    print("Number of unique output tokens:"
    , num_decoder_tokens)
    
    max_input_seq_length = max([len(txt) for txt in input_words])
    max_output_seq_length = max([len(txt) for txt in output_words])	
    print("Max sequence length for inputs:"
    , max_input_seq_length)
    print("Max sequence length for outputs:"
    , max_output_seq_length)
        

    encoder_input_data = np.zeros((len(input_words), max_input_seq_length, num_encoder_tokens), dtype="float32")
    decoder_input_data = np.zeros((len(input_words), max_output_seq_length, num_decoder_tokens), dtype="float32")
    decoder_target_data = np.zeros((len(input_words), max_output_seq_length, num_decoder_tokens), dtype="float32")

    for i, (input_text, target_text) in enumerate(zip(input_words, output_words)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index_map[char]] = 1.0
        encoder_input_data[i, t + 1 :, input_token_index_map[" "]] = 1.0
        for t, char in enumerate(target_text):            
            decoder_input_data[i, t, output_token_index_map[char]] = 1.0
            if t > 0:                
                decoder_target_data[i, t - 1, output_token_index_map[char]] = 1.0
        decoder_input_data[i, t + 1 :, output_token_index_map[" "]] = 1.0
        decoder_target_data[i, t:, output_token_index_map[" "]] = 1.0
    
    return encoder_input_data, decoder_input_data, decoder_target_data


In [25]:
import numpy as np
import pandas as pd
import os

datapath = "./dakshina_dataset_v1.0"
dataBase = DataProcessor(datapath) 

Number of samples: 44204
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21
Number of samples: 4358
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 18
Max sequence length for outputs: 14
Number of samples: 4502
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 16
Max sequence length for outputs: 15


In [29]:
import os
import wandb
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model, Sequential,  Model
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Activation, LSTM, SimpleRNN, GRU, TimeDistributed

class SeqToSeqTranslation():

  def __init__(self, modelConfigDict, input_token_index_map, output_token_index_map, using_pretrained_model = False):
      self.latentDim = modelConfigDict["latentDim"]
      self.cell_type = modelConfigDict["cell_type"]
      self.numEncoders = modelConfigDict["numEncoders"]
      self.numDecoders = modelConfigDict["numDecoders"]
      self.hidden = modelConfigDict["hidden"]
      self.dropout = modelConfigDict["dropout"]
      self.input_token_index_map = input_token_index_map
      self.output_token_index_map = output_token_index_map
      

  def build_model(self):       
      if self.cell_type == "RNN":
          # Encoder
          encoder_inputs = Input(shape=(None, len(self.input_token_index_map)))
          encoder_outputs = encoder_inputs
          for i in range(1, self.numEncoders + 1):
              encoder = SimpleRNN(self.latentDim, return_state=True, return_sequences=True, dropout=self.dropout)
              encoder_outputs, state = encoder(encoder_inputs)
          encoder_states = [state]

          # Decoder
          decoder_inputs = Input(shape=(None, len(self.output_token_index_map)))
          decoder_outputs = decoder_inputs
          for i in range(1, self.numDecoders + 1):
              decoder = SimpleRNN(self.latentDim, return_sequences=True, return_state=True,dropout=self.dropout)
              decoder_outputs, _ = decoder(decoder_inputs, initial_state=encoder_states)

          # dense
          hidden = Dense(self.hidden, activation="relu")
          hidden_outputs = hidden(decoder_outputs)
          decoder_dense = Dense(len(self.output_token_index_map), activation="softmax")
          decoder_outputs = decoder_dense(hidden_outputs)
          model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
          
          return model
      
      elif self.cell_type == "LSTM":
          # encoder
          encoder_inputs = Input(shape=(None, len(self.input_token_index_map)))
          encoder_outputs = encoder_inputs
          for i in range(1, self.numEncoders + 1):
              encoder = LSTM(self.latentDim, return_state=True, return_sequences=True, dropout=self.dropout)
              encoder_outputs, state_h, state_c = encoder(encoder_outputs)
          encoder_states = [state_h, state_c]

          # decoder
          decoder_inputs = Input(shape=(None, len(self.output_token_index_map)))
          decoder_outputs = decoder_inputs
          for i in range(1, self.numDecoders + 1):
              decoder = LSTM(self.latentDim, return_state=True, return_sequences=True, dropout=self.dropout)
              decoder_outputs, _, _ = decoder(decoder_outputs, initial_state=encoder_states)

          # dense
          hidden = Dense(self.hidden, activation="relu")
          hidden_outputs = hidden(decoder_outputs)
          decoder_dense = Dense(len(self.output_token_index_map), activation="softmax")
          decoder_outputs = decoder_dense(hidden_outputs)
          model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
          
          return model
      
      elif self.cell_type == "GRU":
          # encoder
          encoder_inputs = Input(shape=(None, len(self.input_token_index_map)))
          encoder_outputs = encoder_inputs
          for i in range(1, self.numEncoders + 1):
              encoder = GRU(self.latentDim, return_state=True, return_sequences=True, dropout=self.dropout)
              encoder_outputs, state = encoder(encoder_inputs)
          encoder_states = [state]

          # decoder
          decoder_inputs = Input(shape=(None, len(self.output_token_index_map)))
          decoder_outputs = decoder_inputs
          for i in range(1, self.numDecoders + 1):
              decoder = GRU(self.latentDim, return_sequences=True, return_state=True, dropout=self.dropout)
              decoder_outputs, _ = decoder(decoder_inputs, initial_state=encoder_states)

          # dense
          hidden = Dense(self.hidden, activation="relu")
          hidden_outputs = hidden(decoder_outputs)
          decoder_dense = Dense(len(self.output_token_index_map), activation="softmax")
          decoder_outputs = decoder_dense(hidden_outputs)
          model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
          
          return model

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [23]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf


from tensorflow.keras import Input, Model
from tensorflow.keras.layers import RNN, LSTM, GRU, Dense
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

import wandb
from wandb.keras import WandbCallback


def train():    

    config_defaults = {
        "cell_type": "GRU",
        "latentDim": 256,
        "hidden": 128,
        "optimiser": "adam",
        "numEncoders": 3,
        "numDecoders": 1,
        "dropout": 0.1,
        "epochs": 20,
        "batch_size": 32,
    }

    
    wandb.init(config=config_defaults,  project="CS6910_DeepLearning_Assignment3", entity="cs21z032_cs22z005")
    config = wandb.config
    
    wandb.run.name = (str(config.cell_type)+ dataBase.input_language+ str(config.numEncoders)+ "_"+ dataBase.output_language+ "_"+ str(config.numDecoders)+ "_"+ config.optimiser+ "_"+ str(config.epochs)+ "_"+ str(config.dropout) + "_"+ str(config.batch_size)+ "_"+ str(config.latentDim))
    wandb.run.save()

    modelInit = SeqToSeqTranslation(config,input_token_index_map=dataBase.input_token_index_map, output_token_index_map=dataBase.output_token_index_map)
    
    model = modelInit.build_model()
    
    model.summary()

    model.compile(optimizer=config.optimiser,loss="categorical_crossentropy",metrics=["accuracy"])

    earlystopping = EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience=5, verbose=2, mode="auto")

    model.fit([dataBase.train_encoder_input, dataBase.train_decoder_input],dataBase.train_decoder_target,batch_size=config.batch_size,epochs=config.epochs,validation_data=([dataBase.val_encoder_input, dataBase.val_decoder_input], dataBase.val_decoder_target),callbacks=[earlystopping, WandbCallback()])

    model.save(os.path.join("./TrainedModels", wandb.run.name))
    #model.save(os.path.join("/content/gdrive/MyDrive/CS6910_Assignment3/BestModelWithoutAttention", wandb.run.name))
    wandb.finish()  
    

In [ ]:
sweep_config = {
    "name": "Bayesian Sweep",
    "method": "bayes",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        
        "batch_size": {"values": [32, 64]},

        "cell_type": {"values": ["RNN", "GRU", "LSTM"]},

        "optimiser": {"values": ["rmsprop", "adam"]},

        "dropout": {"values": [0.1, 0.2, 0.3]},

        "epochs": {"values": [5,10,15, 20]},

        "hidden": {"values": [128, 64]},

        "latentDim": {"values": [256]},

        "numEncoders": {"values": [1, 2, 3]},        

        "numDecoders": {"values": [1, 2, 3]},  
    },
}

# sweep_config = {
#     "name": "Bayesian Sweep without attention",
#     "method": "bayes",
#     "metric": {"name": "val_accuracy", "goal": "maximize"},
#     "parameters": {
        
#         "cell_type": {"values": ["GRU"]},
        
#         "latentDim": {"values": [256]},
        
#         "hidden": {"values": [128]},
        
#         "optimiser": {"values": ["adam"]},
        
#         "numEncoders": {"values": [3]},
        
#         "numDecoders": {"values": [1]},
        
#         "dropout": {"values": [0.1]},
        
#         "epochs": {"values": [20]},
        
#         "batch_size": {"values": [32]},
#     },
# }

sweep_id = wandb.sweep(sweep_config, project="CS6910_DeepLearning_Assignment3", entity="cs21z032_cs22z005")

wandb.agent(sweep_id, train, count = 200)

In [6]:
import numpy as np
import pandas as pd
import os

from tensorflow.keras import Input, Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, Activation, LSTM, SimpleRNN, GRU, TimeDistributed, Concatenate


import wandb
from wandb.keras import WandbCallback

import tensorflow as tf


model = load_model("/content/gdrive/MyDrive/CS6910_Assignment3/BestModelWithoutAttention/GRUen3_hi_1_adam_20_0.1_32_256")

config_best = {"cell_type": "GRU", "latentDim": 256," hidden": 128, "optimiser": "adam", "numEncoders": 3, "numDecoders": 1, "dropout": 0.1, "epochs": 20, "batch_size": 32}


def test_model(model):    
    wandb.init(config=config_best,  project="CS6910_DeepLearning_Assignment3", entity="cs21z032_cs22z005")
    config = wandb.config
    wandb.run.name = ("Inference_" + str(config.cell_type)+ dataBase.source_lang+ str(config.numEncoders)+ "_"+ dataBase.target_lang+ "_"+ str(config.numDecoders)+ "_"+ config.optimiser+ "_"+ str(config.epochs)+ "_"+ str(config.dropout) + "_"+ str(config.batch_size)+ "_"+ str(config.latentDim))
    wandb.run.save()


    if config.cell_type == "LSTM":
        # encoder
        encoder_inputs = model.input[0]
        
        if config.numEncoders == 1:
            encoder_outputs, state_h_enc, state_c_enc = model.get_layer(name = "lstm").output 
        else:           
            encoder_outputs, state_h_enc, state_c_enc = model.get_layer(name = "lstm_"+ str(config.numEncoders-1)).output

        encoder_states = [state_h_enc, state_c_enc]
        encoder_model = Model(encoder_inputs, encoder_states)

        # decoder
        decoder_inputs = model.input[1]        
        decoder_state_input_c = Input(shape=(config.latentDim,), name="input_4")
        decoder_state_input_h = Input(shape=(config.latentDim,), name="input_3")
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        
        decoder_lstm = model.layers[-3]
        decoder_outputs, state_h_dec, state_c_dec = decoder_lstm( decoder_inputs, initial_state=decoder_states_inputs )
        decoder_states = [state_h_dec, state_c_dec]

        decoder_dense = model.layers[-2]
        decoder_outputs = decoder_dense(decoder_outputs)
        
        decoder_dense = model.layers[-1]
        decoder_outputs = decoder_dense(decoder_outputs)

        decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    elif config.cell_type == "GRU" or config.cell_type == "RNN":
        # encoder
        encoder_inputs = model.input[0]
        if config.cell_type == "GRU":
            if config.numEncoders == 1:
                encoder_outputs, state = model.get_layer(name = "gru").output
            else:
                encoder_outputs, state = model.get_layer(name = "gru_"+ str(config.numEncoders-1)).output
        else:
            if config.numEncoders == 1:
                encoder_outputs, state = model.get_layer(name = "simple_rnn").output
            else:
                encoder_outputs, state = model.get_layer(name = "simple_rnn_"+ str(config.numEncoders-1)).output

        encoder_states = [state]
        encoder_model = Model(encoder_inputs, encoder_states)

        # decoder
        decoder_inputs = model.input[1]
        decoder_state = Input(shape=(config.latentDim,), name="input_3")
        decoder_states_inputs = [decoder_state]
        decoder_gru = model.layers[-3]
        (decoder_outputs, state,) = decoder_gru(decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state]
        decoder_dense = model.layers[-2]
        decoder_outputs = decoder_dense(decoder_outputs)
        decoder_dense = model.layers[-1]
        decoder_outputs = decoder_dense(decoder_outputs)
        decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    def decode_sequence(input_seq):
      states_value = encoder_model.predict(input_seq)
      output_sequence = np.zeros((1, 1, len(dataBase.output_token_index_map)))
      output_sequence[0, 0, dataBase.output_token_index_map["\n"]] = 1.0

      stop_condition = False
      decoded_sentence = ""
      output_index_token_map = dict((i, char) for char, i in dataBase.output_token_index_map.items())
      while not stop_condition:
          if config.cell_type == "LSTM":
              output_tokens, h, c = decoder_model.predict([output_sequence] + states_value)
          elif config.cell_type == "RNN" or config.cell_type == "GRU":
              states_value = states_value[0].reshape((1, 256))
              output_tokens, h = decoder_model.predict([output_sequence] + [states_value])
                    
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = output_index_token_map[sampled_token_index]
          decoded_sentence += sampled_char
          
          if sampled_char == "\n" or len(decoded_sentence) > 25:
              stop_condition = True

          output_sequence = np.zeros((1, 1, len(dataBase.output_token_index_map)))
          output_sequence[0, 0, sampled_token_index] = 1.0

          if config.cell_type == "LSTM":
              states_value = [h, c]
          elif config.cell_type == "RNN" or config.cell_type == "GRU":
              states_value = [h]
      return decoded_sentence

    acc = 0
    input_text = []
    original = []
    predictions = []
    
    for i, row in dataBase.test.iterrows():
        input_seq = dataBase.test_encoder_input[i : i + 1]
        decoded_sentence = decode_sequence(input_seq)
        predictions.append(decoded_sentence)
    
        original_tokens = [dataBase.output_token_index_map[x] for x in row["tgt"]]
        predicted_tokens = [dataBase.output_token_index_map[x] for x in decoded_sentence.rstrip("\n")]
            
        input_text.append(row['src'])
        original.append(row['tgt'])
        
        if original_tokens == predicted_tokens:
            acc += 1

        if i % 100 == 0:
            print(f"Finished {i} examples")
            print(f"Input text: {row['src']}")
            print(f"Original: {row['tgt']}")
            print(f"Predicted: {decoded_sentence}")
            print(f"Accuracy: {acc / (i+1)}")
            print(original_tokens)
            print(predicted_tokens)
            

    print(f'Test Accuracy: {acc}')
    wandb.log({'test_accuracy': acc / len(dataBase.test)})
    wandb.finish()
    return acc / len(dataBase.test), input_text, original, predictions


acc,input_text, original, predictions = test_model(model)
data = [{"input":input_text[i], "true": original[i], "predicted": predictions[i]} for i in range(len(input_text))] 
test_predictions = pd.DataFrame(data)
test_predictions.to_csv('/content/gdrive/MyDrive/CS6910_Assignment3/BestModelWithoutAttention/predictions_vanilla.csv', index=False, sep='\t')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Finished 0 examples
Source: ank
Original: अंक
Predicted: एंक

Accuracy: 0.0
[5, 3, 17]
[12, 3, 17]
Finished 100 examples
Source: anukulata
Original: अनुकूलता
Predicted: अनुकुलता

Accuracy: 0.1782178217821782
[5, 36, 54, 17, 55, 44, 32, 51]
[5, 36, 54, 17, 54, 44, 32, 51]
Finished 200 examples
Source: avaru
Original: अवरु
Predicted: अवरू

Accuracy: 0.19900497512437812
[5, 45, 43, 54]
[5, 45, 43, 55]
Finished 300 examples
Source: aabru
Original: आबरू
Predicted: अब्रू

Accuracy: 0.2691029900332226
[6, 39, 43, 55]
[5, 39, 63, 43, 55]
Finished 400 examples
Source: inhaletion
Original: इनहेलेशन
Predicted: इंहालियों

Accuracy: 0.27680798004987534
[7, 36, 49, 58, 44, 58, 46, 36]
[7, 3, 49, 51, 44, 52, 42, 61, 3]
Finished 500 examples
Source: umesh
Original: उमेश
Predicted: उमेश

Accuracy: 0.27944111776447106
[9, 41, 58, 46]
[9, 41, 58, 46]
Finished 600 examples
Source: asphalt
Original: एस्फाल्ट
Predicted: अस्पालत

Accuracy: 0.2795341098169717
[12, 48, 63, 38, 51, 44, 63, 27]
[5, 48, 63, 37, 5

test_accuracy,▁
test_accuracy,0.31941
